In [24]:
! lamin load scprint

^C
Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/scprint/bin/lamin", line 5, in <module>
    from lamin_cli.__main__ import main
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/lamin_cli/__main__.py", line 11, in <module>
    import rich_click as click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/__init__.py", line 73, in <module>
    from . import rich_click as rich_click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/rich_click.py", line 6, in <module>
    import rich.columns
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/columns.py", line 7, in <module>
    from .console import Console, ConsoleOptions, RenderableType, RenderResult
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/console.py", line 57, in <module>
    from .markup import render as render_markup
  File "<frozen importlib._bootstra

In [1]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Global seed set to 42


💡 connected lamindb: jkobject/scprint


In [2]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_labels = [
    "cell_type_ontology_term_id", #1
    # "tissue_ontology_term_id",
    "disease_ontology_term_id", # 2
#    "development_stage_ontology_term_id",
    "assay_ontology_term_id", #3
    'self_reported_ethnicity_ontology_term_id', #4
]
labels_to_pred = hierarchical_labels+[
    'sex_ontology_term_id', #5
    "organism_ontology_term_id", #6
]
all_labels = labels_to_pred+[
    #'dataset_id',
    'cell_culture',
  #  "heat_diff",
  #  "total_counts",
  # "nnz",
  #  "dpt_group",
]

gene_emb = '../data/main/gene_embeddings.parquet'
d_model=128

In [3]:
datamodule = DataModule(
    collection_name="some",
    gene_embeddings=gene_emb,
    all_labels=all_labels,
    hierarchical_labels=hierarchical_labels,
    organisms=["NCBITaxon:9606", "NCBITaxon:10090"],
    how="random expr",
    max_len=1200,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category 
    weight_scaler=10,
    label_to_weight=labels_to_pred,
    label_to_pred=labels_to_pred,
    batch_size=64,
    num_workers=16,
    #train_oversampling=2,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    test_split=0.05)
testfiles = datamodule.setup()

won't do any check but we recommend to have your dataset coming from local storage

100.0% are aligned
total dataset size is 134.0236856 Gb
---
dataset contains:
     5884587 cells
     127057 genes
     7 labels
     6 clss_to_pred
     4 hierarchical_clss
     2 organisms
dataset contains 270 classes to predict

seeing a string: loading gene positions as biomart parquet file
these files will be considered test datasets:
    /home/ml4ig1/scprint/.lamindb/R4ZHoQegxXdSFNFY5LGe.h5ad
    /home/ml4ig1/scprint/.lamindb/SHV11AEetZOms4Wh7Ehb.h5ad
    /home/ml4ig1/scprint/.lamindb/V6DPJx8rP3wWRQ43LMHb.h5ad
    /home/ml4ig1/scprint/.lamindb/Gz5G2ETTEuuRDgwm7brA.h5ad
    /home/ml4ig1/scprint/.lamindb/YyBdEsN89p2aF4xJY1CW.h5ad
    /home/ml4ig1/scprint/.lamindb/SO5yBTUDBgkAmz0QbG8K.h5ad
    /home/ml4ig1/scprint/.lamindb/r4iCehg3Tw5IbCLiCIbl.h5ad
perc test:  0.041448618229282704


In [4]:
datamodule.dataset.encoder

{'cell_type_ontology_term_id': {'CL:0005012': 0,
  'CL:0000038': 218,
  'CL:0000763': 219,
  'CL:0008034': 220,
  'CL:1000692': 1,
  'CL:0002063': 2,
  'CL:0000837': 221,
  'CL:0000623': 222,
  'CL:0000630': 223,
  'CL:0002355': 3,
  'CL:0005009': 224,
  'CL:4023070': 4,
  'CL:0000169': 5,
  'CL:1001107': 6,
  'CL:0000878': 225,
  'CL:0000751': 7,
  'CL:0001029': 8,
  'CL:4023041': 9,
  'CL:0000186': 10,
  'CL:0000545': 11,
  'CL:0000057': 226,
  'CL:0002303': 12,
  'CL:1000223': 13,
  'CL:0000084': 227,
  'CL:0000077': 228,
  'CL:1001433': 14,
  'CL:0002250': 229,
  'CL:0000653': 15,
  'CL:0000827': 230,
  'CL:0001049': 16,
  'CL:0000954': 17,
  'CL:0009009': 18,
  'CL:0002377': 19,
  'CL:0000148': 20,
  'CL:0002144': 21,
  'CL:4030026': 22,
  'CL:0000449': 23,
  'CL:1000343': 24,
  'CL:0019032': 231,
  'CL:0000785': 232,
  'CL:4023051': 25,
  'CL:0000558': 26,
  'CL:0011007': 27,
  'CL:0002010': 28,
  'CL:1001111': 29,
  'CL:0005010': 233,
  'CL:0000875': 234,
  'CL:0000232': 30,
  '

In [ ]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test", save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

#tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
#tlogger.log_graph(model)


2024-03-14 09:16:21,556:ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin


NameError: name 'model' is not defined

In [ ]:
#from lightning.pytorch.profilers import PyTorchProfiler
#pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [12]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(
    do_denoise=True, 
    noise=[0.4],
    do_cce=True, 
    cce_sim=0.6, 
    do_ecs=True, 
    ecs_threshold = 0.4, 
    ecs_scale = 0.05,
    class_scale = 0.08,
    do_mvc=False, 
    do_adv_cls=True,
    do_next_tp=False, 
    mask_ratio=[0.3], 
    warmup_duration= 500, 
    weight_decay= 0.01, 
    fused_adam= True,
    lr_patience= 1)
es = EarlyStopping(patience=2, monitor='val_loss')
swa = StochasticWeightAveraging(swa_lrs= 0.01)
lrm = LearningRateMonitor(logging_interval="step")
#lrf = LearningRateFinder(mode="exponential",)
# TODO: to check that the class hierarchy are really ordered from 1-2-3-4... as well (oredered dict)
trainer = Trainer(precision="16-mixed", gradient_clip_val=100, max_time={"hours": 2}, limit_train_batches=50, limit_val_batches=10, callbacks=[chckp, es, lrm,  swa, trainingmode], accumulate_grad_batches=1, reload_dataloaders_every_n_epochs=1)#, logger=tlogger) #detect_anomaly=True, fast_dev_run=20, overfit_batches=10, limit_train_batches=1, limit_val_batches=0
#logger=wandb_logger,

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
model.labels

['cell_type_ontology_term_id',
 'disease_ontology_term_id',
 'assay_ontology_term_id',
 'self_reported_ethnicity_ontology_term_id',
 'sex_ontology_term_id',
 'organism_ontology_term_id']

In [13]:
# sanity. should be overfiting.
trainer = Trainer(precision="16-mixed", max_epochs=1000, limit_val_batches=0, check_val_every_n_epoch=1000, log_every_n_steps=1000, detect_anomaly=True, overfit_batches=30, gradient_clip_val=100,
reload_dataloaders_every_n_epochs=1000) #logger=wandb_logger) limit_train_batches=1

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
model = scPrint(
    genes = datamodule.genes,
    d_model = d_model,
    nhead = 4,
    nlayers = 4,
    layers_cls = [d_model],
    labels = datamodule.labels,
    cls_hierarchy = datamodule.cls_hierarchy,
    dropout= 0.1,
    transformer = "flash",
    precpt_gene_emb = gene_emb,
    gene_pos_enc = datamodule.gene_pos,
    mvc_decoder = "inner product",
    label_decoders = datamodule.decoders,
    fused_dropout_add_ln = False,
    num_batch_labels = datamodule.num_datasets,
    checkpointing=True,
    prenorm=True,
    num_heads_kv=None,
)

model.do_denoise = True
model.noise = [.5]
model.do_cce = True
model.do_mvc = False
model.do_adv_cls = False
model.do_generate = True
model.weight_decay = 1e-4
model.fused_adam = True
model.lr_patience = 1
model.optim = "adamW"

In [33]:
trainer.fit(model, datamodule=datamodule)

these files will be considered test datasets:
    /home/ml4ig1/scprint/.lamindb/R4ZHoQegxXdSFNFY5LGe.h5ad
    /home/ml4ig1/scprint/.lamindb/SHV11AEetZOms4Wh7Ehb.h5ad
    /home/ml4ig1/scprint/.lamindb/V6DPJx8rP3wWRQ43LMHb.h5ad
    /home/ml4ig1/scprint/.lamindb/Gz5G2ETTEuuRDgwm7brA.h5ad
    /home/ml4ig1/scprint/.lamindb/YyBdEsN89p2aF4xJY1CW.h5ad
    /home/ml4ig1/scprint/.lamindb/SO5yBTUDBgkAmz0QbG8K.h5ad
    /home/ml4ig1/scprint/.lamindb/r4iCehg3Tw5IbCLiCIbl.h5ad
perc test:  0.041448618229282704


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                            | Type                         | Params
----------------------------------------------------------------------------------
0  | gene_encoder                    | GeneEncoder                  | 5.7 M 
1  | expr_encoder                    | ContinuousValueEncoder       | 17.0 K
2  | pos_encoder                     | PositionalEncoding           | 0     
3  | label_encoder                   | CategoryValueEncoder         | 1.0 K 
4  | depth_encoder                   | ContinuousValueEncoder       | 17.0 K
5  | norm_and_dropout                | Sequential                   | 256   
6  | transformer                     | FlashTransformerEncoder      | 793 K 
7  | expr_decoder                    | ExprDecoder                  | 33.7 K
8  | cls_decoders                    | ModuleDict                   | 135 K 
9  | grad_reverse_discriminator_loss | AdversarialDiscriminatorLoss | 45.1 K
10 | mvc_decoder           

Sanity Checking: 0it [00:00, ?it/s]

> /home/ml4ig1/Documents code/scPRINT/scprint/model/utils.py(54)make_adata()
     52 
     53         pdb./home/ml4ig1/Documents code/scPRINT/scprint/model/utils.py(54)make_adata()
     52 
     53         pdb.set_trace()
---> 54         obs = np.array(
     55             [
     56                 [label_decoders[labels[i]][n] for n in name]



In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [ ]:
#wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()